In [ ]:
from IPython.core.display import display, HTML
import os
import errno
from multiprocessing.pool import Pool
import sys
sys.path.append('../')
from Utils.download import *
import pandas as pd

%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

out_folder = 'D:/data/HPA/all/external/'
print('Done.')

#### Following code is based on https://www.kaggle.com/c/human-protein-atlas-image-classification/discussion/69984

In [ ]:
# Parameters
process_num = 24
image_size = (512, 512)
url = 'http://v18.proteinatlas.org/images/'
csv_path =  "D:/data/HPA/all/external.csv"
save_dir = out_folder

# Create the directory to save the images in case it doesn't exist
try:
    os.makedirs(save_dir)
except OSError as exc:
    if exc.errno != errno.EEXIST:
        raise
    pass

print('Parent process %s.' % os.getpid())
print("Reading external csv..")
img_list = pd.read_csv(csv_path)['Id']
list_len = len(img_list)
p = Pool(process_num)

print("Starting processes. Listlength = ",list_len)
for i in range(process_num):
    start = int(i * list_len / process_num)
    end = int((i + 1) * list_len / process_num)
    process_images = img_list[start:end]
    p.apply_async(
        download, args=(str(i), process_images, url, save_dir, image_size)
    )
print('Waiting for all subprocesses done...')
p.close()
p.join()
print('All subprocesses done.')